
# Cryptocurrency Price Prediction & Trading Strategy using Machine Learning

This project fetches real-time cryptocurrency data from CoinMarketCap's API and applies machine learning techniques to predict the top 10% price movers in the next hour. The model leverages financial indicators, sentiment analysis, and technical analysis to identify trading opportunities.

## Key Features:
- Fetches live cryptocurrency data (price, volume, volatility) using the CoinMarketCap API.
- Uses sentiment analysis from the VADER model to gauge public opinion on cryptocurrencies.
- Implements technical indicators such as MACD, RSI, and SMA for feature engineering.
- Applies an ensemble machine learning model (Random Forest & Logistic Regression) with SMOTE to handle class imbalance.
- Simulates a trading strategy based on model predictions, incorporating dynamic thresholds and risk management techniques.



In [4]:
!pip install ta vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [5]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score
from ta.momentum import RSIIndicator
from ta.trend import MACD, SMAIndicator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# API Configuration
API_KEY = "4a48b2a6-d0cd-4f4c-879c-b74c20ad70a0"
BASE_URL = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": API_KEY,
}
parameters = {
    "start": "1",
    "limit": "100",  # Fetch top 100 cryptocurrencies
    "convert": "USD",
}

# Fetch Cryptocurrency Data
response = requests.get(BASE_URL, headers=headers, params=parameters)
if response.status_code == 200:
    data = response.json()["data"]
    df = pd.DataFrame(data)
    processed_df = df[["name", "symbol", "quote"]].copy()
    processed_df["price_change_1h"] = processed_df["quote"].apply(lambda x: x["USD"]["percent_change_1h"])
    processed_df["volume_24h"] = processed_df["quote"].apply(lambda x: x["USD"]["volume_24h"])
    processed_df["price"] = processed_df["quote"].apply(lambda x: x["USD"]["price"])
    processed_df.drop(columns=["quote"], inplace=True)
else:
    raise Exception(f"Error: Unable to fetch data (Status Code: {response.status_code})")

# Sentiment Analysis Function
def get_sentiment(coin_name):
    analyzer = SentimentIntensityAnalyzer()
    sample_text = f"{coin_name} is trending on Twitter with many positive mentions."
    return analyzer.polarity_scores(sample_text)["compound"]

processed_df["sentiment"] = processed_df["name"].apply(get_sentiment)

# Feature Engineering
def generate_features(df, lookback=3):
    df["volatility"] = df["price_change_1h"].rolling(lookback).std()
    df["volume_change"] = df["volume_24h"].pct_change()
    df["relative_volume"] = df["volume_24h"] / df["volume_24h"].mean()
    df["sentiment_trend"] = df["sentiment"].rolling(lookback).mean()
    df["sma_short"] = SMAIndicator(df["price"], window=5).sma_indicator()
    df["sma_long"] = SMAIndicator(df["price"], window=20).sma_indicator()
    df["momentum"] = df["sma_short"] - df["sma_long"]
    macd = MACD(df["price"])
    df["macd"] = macd.macd()
    df["macd_signal"] = macd.macd_signal()
    df.dropna(inplace=True)
    return df

processed_df = generate_features(processed_df)

# Define Target Variable (Top 10% Movers)
threshold = np.percentile(processed_df["price_change_1h"], 90)
processed_df["is_top_mover_next"] = (processed_df["price_change_1h"] >= threshold).astype(int)
processed_df.dropna(inplace=True)

# Prepare Features and Target
features = [
    "volatility", "volume_change", "relative_volume", "sentiment", "sentiment_trend",
    "momentum", "macd", "macd_signal"
]
X = processed_df[features]
y = processed_df["is_top_mover_next"]

# Normalize Features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Handle Class Imbalance with SMOTE
if y.sum() >= 2:
    smote = SMOTE(random_state=42, k_neighbors=min(5, int(y.sum() - 1)))
    X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
else:
    print("Not enough samples for SMOTE; proceeding without oversampling.")
    X_resampled, y_resampled = X_scaled, y

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train Ensemble Model (Random Forest + Logistic Regression)
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight="balanced", random_state=42)
lr_model = LogisticRegression(C=0.5, max_iter=500)
ensemble_model = VotingClassifier(estimators=[("rf", rf_model), ("lr", lr_model)], voting="soft")
ensemble_model.fit(X_train, y_train)

# Model Evaluation
y_prob = ensemble_model.predict_proba(X_test)[:, 1]
dynamic_threshold = np.percentile(y_prob, 50)
y_pred = (y_prob > dynamic_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)

print(f"Model Accuracy: {accuracy * 100:.2f}%")
print(f"Model Precision: {precision * 100:.2f}%")
print(f"Model Recall: {recall * 100:.2f}%")

# Trading Strategy Simulation
def simulate_trading(probabilities, price_changes, coin_names, initial_balance=1000, volatility=None):
    balance = initial_balance
    trades = []
    for prob, price_change, coin_name, vol in zip(probabilities, price_changes, coin_names, volatility):
        if prob > dynamic_threshold:
            investment = balance * prob
            stop_loss = max(-5, -vol * 1.5)
            profit_take = 10
            profit = investment * (price_change / 100)
            if profit / investment * 100 < stop_loss:
                profit = investment * (stop_loss / 100)
            elif profit / investment * 100 > profit_take:
                profit = investment * (profit_take / 100)
            balance += profit
            trades.append((coin_name, prob, investment, profit))
    return balance, trades

# Simulate Trading
test_prices = processed_df.iloc[-len(y_test):]["price_change_1h"].values
volatility = processed_df.iloc[-len(y_test):]["volatility"].values
coin_names = processed_df.iloc[-len(y_test):]["name"].values
final_balance, trades = simulate_trading(y_prob, test_prices, coin_names, volatility=volatility)

print(f"\nFinal Balance: ${final_balance:.2f}")
print("\nTrades:")
for trade in trades:
    coin_name, prob, investment, profit = trade
    print(f"Coin: {coin_name}, Confidence: {prob:.2f}, Investment: ${investment:.2f}, Profit: ${profit:.2f}")


Model Accuracy: 91.67%
Model Precision: 83.33%
Model Recall: 100.00%

Final Balance: $1035.23

Trades:
Coin: Tezos, Confidence: 0.71, Investment: $712.89, Profit: $7.57
Coin: Ethereum Name Service, Confidence: 0.81, Investment: $812.73, Profit: $6.84
Coin: Nexo, Confidence: 0.81, Investment: $824.80, Profit: $0.61
Coin: BitTorrent [New], Confidence: 0.78, Investment: $792.67, Profit: $3.24
Coin: Flow, Confidence: 0.72, Investment: $736.23, Profit: $3.01
Coin: Pyth Network, Confidence: 0.77, Investment: $784.30, Profit: $3.49
Coin: Neo, Confidence: 0.74, Investment: $759.66, Profit: $3.82
Coin: PAX Gold, Confidence: 0.75, Investment: $774.72, Profit: $0.27
Coin: Helium, Confidence: 0.59, Investment: $604.62, Profit: $4.13
Coin: MultiversX, Confidence: 0.64, Investment: $659.18, Profit: $-0.70
Coin: Ronin, Confidence: 0.37, Investment: $382.23, Profit: $1.57
Coin: dogwifhat, Confidence: 0.67, Investment: $695.24, Profit: $1.38
